# LangChain + OpenAI (Updated) — Prompts 

This notebook demonstrates **modern LangChain imports** (`langchain-openai`) and a variety of **prompting patterns** 
- Basic Q&A
- Summarization (with constraints)
- Classification (labels)
- Information extraction (JSON)
- Rewrite / tone change
- Few-shot prompting
- Prompt templates
- Simple evaluation + retry
- Batch processing over a list / DataFrame

> **Important:** Never hardcode API keys in notebooks. Use `.env` or a secure prompt.


## 0) Install (run once)

If you see version-mismatch errors (like `ensure_id` import issues), do a clean reinstall and **restart kernel**.


In [ ]:
!pip uninstall -y langchain langchain-core langchain-community langchain-openai langchain-cohere >/dev/null 2>&1
!pip install -U pip >/dev/null 2>&1
!pip install -U "langchain-core>=1.2.7" "langchain>=1.0.0" "langchain-openai" "python-dotenv" pandas >/dev/null 2>&1
print("Installed / upgraded packages. Now restart the kernel (Kernel -> Restart).")

## 1) Setup API key safely

### Option A: `.env` file (recommended)

Create a `.env` file in the same folder:
```env
OPENAI_API_KEY=your_key_here
```

### Option B: prompt for key 


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Fallback: prompt if not already set
if not os.environ.get("OPENAI_API_KEY"):
    import getpass
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI_API_KEY: ")

print("OPENAI_API_KEY set:", "YES" if os.environ.get("OPENAI_API_KEY") else "NO")

## 2) Create an LLM client (ChatOpenAI)

- `temperature=0` for more deterministic answers
- Choose a model you have access to (example: `gpt-4o-mini`)


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# A quick smoke test
resp = llm.invoke("Give me a 1-line definition of Machine Learning.")
print(resp.content)

## 3) Prompt Example: Basic Q&A with constraints

Teach students to add constraints:
- output length
- formatting
- audience level


In [ ]:
prompt = '''
Explain "overfitting" to a beginner.
Constraints:
- Exactly 3 bullet points
- Each bullet <= 12 words
- No jargon
'''
print(llm.invoke(prompt).content)

## 4) Prompt Example: Summarization (with structure)

Summarize text into a fixed template so outputs are consistent.


In [ ]:
text = '''
LangChain is a framework for building applications with LLMs. It provides abstractions for prompts,
models, tools, agents, memory, and retrieval. Recent versions split provider integrations into
separate packages like langchain-openai, langchain-cohere, etc. This enables faster iteration
and smaller dependency footprints.
'''

prompt = f'''
Summarize the text below using this template:

Title: <5-8 words>
Key Points: (3 bullets)
Action Item: (1 sentence)

Text:
{text}
'''
print(llm.invoke(prompt).content)

## 5) Prompt Example: Classification (label only)

A common enterprise use-case: classify tickets/logs.

Key teaching point: **force a closed set of labels** and ask for **only the label**.


In [ ]:
def classify_log(message: str) -> str:
    prompt = f'''
Return exactly ONE label: INFO, WARNING, CRITICAL

Rules:
- failure/breach/outage/connection failed => CRITICAL
- low disk/memory/latency/unstable => WARNING
- otherwise => INFO

Log: {message}
Label:
'''.strip()
    return llm.invoke(prompt).content.strip().split()[0].upper()

logs = [
    "INFO: Server started successfully",
    "WARNING: Low disk space on /dev/sda1",
    "CRITICAL: Database connection failed",
    "User login succeeded",
]
for l in logs:
    print(l, "->", classify_log(l))

## 6) Prompt Example: Extraction (return JSON only)

Teach students to ask for **valid JSON** only (no commentary).
We can later parse it reliably.


In [ ]:
import json

incident = '''
Incident: INC12345
Short Description: VPN not working for multiple users
Description: Users report VPN disconnects every 2-3 minutes after the latest update.
Impact: High, many teams blocked.
Urgency: High
'''

prompt = f'''
Extract the following fields as VALID JSON (no extra text):
- incident_id
- short_description
- impact (Low/Medium/High)
- urgency (Low/Medium/High)
- likely_root_cause (1 sentence)

Text:
{incident}
'''
raw = llm.invoke(prompt).content.strip()
print(raw)

# Try parsing
data = json.loads(raw)
data

## 7) Prompt Example: Rewrite / tone transformation

Useful for emails, tickets, or policy text.


In [ ]:
text = "Hi team, vpn is down again. fix asap. this is very bad."

prompt = f'''
Rewrite the message in a professional corporate tone.
Constraints:
- 2 sentences
- Include a polite request
- Do not blame anyone

Original:
{text}
'''
print(llm.invoke(prompt).content)

## 8) Few-shot prompting (show examples → get consistent outputs)

Few-shot is very effective for teaching consistent labeling.


In [ ]:
prompt = '''
You are a log severity classifier. Output exactly one label: INFO, WARNING, CRITICAL.

Examples:
Log: "Disk usage is at 92% on server A"
Label: WARNING

Log: "Service cannot connect to database; connection refused"
Label: CRITICAL

Log: "Nightly backup completed successfully"
Label: INFO

Now classify:
Log: "CPU is constantly at 98% for 10 minutes"
Label:
'''.strip()

print(llm.invoke(prompt).content)

## 9) PromptTemplate (parameterized prompts)

This is how you avoid manual string formatting and keep prompts reusable.


In [ ]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(
    '''You are a helpful tutor.
Explain {topic} for a {audience}.
Constraints:
- {bullets} bullet points
- Each bullet <= {max_words} words
- End with one short example
'''
)

prompt = template.format(topic="Pandas DataFrame", audience="beginner", bullets=4, max_words=10)
print(prompt)
print("\n---\n")
print(llm.invoke(prompt).content)

## 10) Simple "guardrail": validate output and retry

Teaching point: you can programmatically check outputs.
Example: ensure label is in allowed set.


In [ ]:
ALLOWED = {"INFO", "WARNING", "CRITICAL"}

def classify_with_retry(message: str, max_tries: int = 3) -> str:
    base_prompt = f'''
Return exactly ONE label: INFO, WARNING, CRITICAL
Log: {message}
Label:
'''.strip()

    prompt = base_prompt
    for i in range(max_tries):
        out = llm.invoke(prompt).content.strip().split()[0].upper()
        if out in ALLOWED:
            return out
        # retry with stronger instruction
        prompt = base_prompt + "\nRemember: output ONLY one of INFO/WARNING/CRITICAL."
    return "UNKNOWN"

tests = [
    "Security breach detected",
    "Low disk space",
    "All good, scheduled job ran",
]
for t in tests:
    print(t, "->", classify_with_retry(t))

## 11) Batch processing with Pandas

A realistic workflow: apply LLM classification to a column.


In [ ]:
import pandas as pd

df = pd.DataFrame({
    "message": [
        "Database connection failed after deploy",
        "Disk usage is 95% on node-2",
        "Service started successfully",
        "Latency spikes observed in API gateway"
    ]
})

df["severity"] = df["message"].apply(classify_with_retry)
df